In [48]:
import joblib
import numpy as np
import tensorflow as tf
import math

from keras.models import Model, Sequential
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Layer,\
                         ZeroPadding2D, Activation, Reshape, Permute, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from PIL import Image
from skimage.measure import compare_ssim as ssim
import matplotlib.pyplot as plt # to plot images
%matplotlib inline

### Consts

In [2]:
train_and_val_dataset_file = 'datasets/dataset-1/train-and-val.pkl'
test_dataset_file = 'datasets/dataset-1/test.pkl'
saved_model_filename = "datasets/dataset-1/test-segnet-{epoch:02d}-{val_dice_coef_accur:.4f}.hdf5"

### Load datasets

In [26]:
X_remaining, Y_remaining, remaining_dataset_desc = joblib.load(train_and_val_dataset_file)
Xte, yte, test_dataset_desc = joblib.load(test_dataset_file) # X and y for test
training_set_index = remaining_dataset_desc['training_set_index']
validation_set_index = remaining_dataset_desc['validation_set_index']

Xtr, ytr = X_remaining[:training_set_index,:], Y_remaining[:training_set_index] # X and y for training
Xva, yva = X_remaining[training_set_index:validation_set_index,:], Y_remaining[training_set_index:validation_set_index] # X and y for validation

print(Xtr.shape)
print(Xva.shape)
print(Xte.shape)
print(ytr.shape)
print(yva.shape)
print(yte.shape)

(5, 128, 128, 1)
(1, 128, 128, 1)
(3, 128, 128, 1)
(5, 128, 128, 1)
(1, 128, 128, 1)
(3, 128, 128, 1)


### Pre processing

In [27]:
# Preprocessing in the training set (mean and sd) and apply it to all sets

full_image_mean_value = Xtr.mean() # mean-value for each pixel of all full images
full_image_sd = Xtr.std() # standard deviation for each pixel of all full images

Xtr = (Xtr - full_image_mean_value) / full_image_sd
Xva = (Xva - full_image_mean_value) / full_image_sd
Xte = (Xte - full_image_mean_value) / full_image_sd

"""Xtr = Xtr.reshape((Xtr.shape[-1], Xtr.shape[1], Xtr.shape[2], Xtr.shape[0]))
ytr = ytr.reshape((ytr.shape[-1], ytr.shape[1], ytr.shape[2], ytr.shape[0]))

Xva = Xva.reshape((Xva.shape[-1], Xva.shape[1], Xva.shape[2], Xva.shape[0]))
yva = yva.reshape((yva.shape[-1], yva.shape[1], yva.shape[2], yva.shape[0]))

Xte = Xte.reshape((Xte.shape[-1], Xte.shape[1], Xte.shape[2], Xte.shape[0]))
yte = yte.reshape((yte.shape[-1], yte.shape[1], yte.shape[2], yte.shape[0]))"""

print(Xtr.shape)
print(Xva.shape)
print(Xte.shape)
print(ytr.shape)
print(yva.shape)
print(yte.shape)

(5, 128, 128, 1)
(1, 128, 128, 1)
(3, 128, 128, 1)
(5, 128, 128, 1)
(1, 128, 128, 1)
(3, 128, 128, 1)


### Pre-configurations

In [34]:
K.set_image_data_format('channels_last')  # TF dimension
_, *input_image_shape, _ = Xtr.shape
input_image_shape = tuple(input_image_shape)
print(input_image_shape)

smooth = 1.

use_dropout = True
use_regularizers = True
dropout_rate = 0.5
number_of_epochs = 1000
batch_size = 32
kernel_size = (5, 5)
initial_volume_size = 64

(128, 128)


### Define Segnet model

In [62]:
# Define model
kernel = 3
filter_size = 64
pad = 1
pool_size = 2
input_height = 128
input_width = 128
nClasses = 2


inputs = Input(shape=(*input_image_shape, 1))
print(inputs.shape)
x = ZeroPadding2D(padding=(pad,pad))(inputs)
print(x.shape)
x = Conv2D(filter_size, (kernel, kernel), padding='valid')(x)
print(x.shape)
x = BatchNormalization()(x)
print(x.shape)
x = Activation('relu')(x)
print(x.shape)
x = MaxPooling2D(pool_size=(pool_size, pool_size))(x)
print(x.shape)

x = ZeroPadding2D(padding=(pad,pad))(x)
x = Conv2D(128, (kernel, kernel), padding='valid')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(pool_size, pool_size))(x)

x = ZeroPadding2D(padding=(pad,pad))(x)
x = Conv2D(256, (kernel, kernel), padding='valid')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(pool_size, pool_size))(x)

x = ZeroPadding2D(padding=(pad,pad))(x)
x = Conv2D(512, (kernel, kernel), padding='valid')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = ZeroPadding2D(padding=(pad,pad))(x)
x = Conv2D(512, (kernel, kernel), padding='valid')(x)
x = BatchNormalization()(x)

x = UpSampling2D(size=(pool_size,pool_size))(x)
x = ZeroPadding2D(padding=(pad,pad))(x)
x = Conv2D(256, (kernel, kernel), padding='valid')(x)
x = BatchNormalization()(x)

x = UpSampling2D(size=(pool_size,pool_size))(x)
x = ZeroPadding2D(padding=(pad,pad))(x)
x = Conv2D(128, (kernel, kernel), padding='valid')(x)
x = BatchNormalization()(x)

x = UpSampling2D(size=(pool_size,pool_size))(x)
x = ZeroPadding2D(padding=(pad,pad))(x)
x = Conv2D(filter_size, (kernel, kernel), padding='valid')(x)
x = BatchNormalization()(x)

x = Conv2D( nClasses , (1, 1), padding='valid')(x)

print(x.shape[-2])
print(x.shape[-1])
print(x.shape[-2] * x.shape[-1] )

print(x.shape)
x = Reshape(( nClasses ,  128 * 128   ), input_shape=( nClasses , x.shape[-2], x.shape[-1]  ))(x)
print(x.shape)
x = Permute((2, 1))(x)
print(x.shape)
x = Activation('softmax')(x)

model = Model(inputs=[inputs], outputs=[x])
model.compile(loss="categorical_crossentropy", optimizer= Adam(lr=1e-5) , metrics=['accuracy'] )

# Options for the model
print("Size of the CNN: %s" % model.count_params())

(?, 128, 128, 1)
(?, 130, 130, 1)
(?, 128, 128, 64)
(?, 128, 128, 64)
(?, 128, 128, 64)
(?, 64, 64, 64)
128
2
256
(?, 128, 128, 2)
(?, 2, 16384)
(?, 16384, 2)
Size of the CNN: 5466178


### Train model

In [63]:
model_checkpoint = ModelCheckpoint(saved_model_filename, monitor='accuracy', save_best_only=True, verbose=1)

history = model.fit(Xtr, ytr, batch_size=batch_size, epochs=number_of_epochs, verbose=2, shuffle=True,
             callbacks=[model_checkpoint], validation_data=(Xva, yva))

ValueError: Error when checking target: expected activation_142 to have 3 dimensions, but got array with shape (5, 128, 128, 1)

### Show model metrics

In [ ]:
x = history.history['dice_coef_accur']
y = history.history['val_dice_coef_accur']
plt.plot(x, label='train')
plt.plot(y, label = 'val')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

x = history.history['loss']
y = history.history['val_loss']
plt.plot(x, label='train')
plt.plot(y, label='val')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### Evaluate the model

In [ ]:
test_loss, accuracy_test = model.evaluate(Xte, yte, verbose=0)
print("Training Accuracy Mean: "+str(np.array(history.history['dice_coef_accur']).mean()))
print("Validation Accuracy Mean: "+str(np.array(history.history['val_dice_coef_accur']).mean()))
print("Test Accuracy Mean: "+str(accuracy_test))

### Predict masks using the trained model

In [ ]:
model.load_weights("datasets/dataset-1/test-6-08-0.0069.hdf5")
test_loss, accuracy_test = model.evaluate(Xte, yte, verbose=0)
print("Test Accuracy Mean: "+str(accuracy_test))
imgs_mask_test = model.predict(Xte, verbose=1)

### Show results

In [ ]:
ncols = 3 # number of columns in final grid of images
nrows = 30 # looking at all images takes some time
_, axes = plt.subplots(nrows, ncols, figsize=(17, 17*nrows/ncols))
for axis in axes.flatten():
    axis.set_axis_off()
    axis.set_aspect('equal')

for k in range(0, nrows):
    im_test_original = Xte[k].reshape(*input_image_shape)
    im_result = imgs_mask_test[k].reshape(*input_image_shape)
    im_ground_truth = yte[k].reshape(*input_image_shape)
    
    axes[k, 0].set_title("Original Test Image")
    axes[k, 0].imshow(im_test_original, cmap='gray')
    
    axes[k, 1].set_title("Ground Truth")
    axes[k, 1].imshow(im_ground_truth, cmap='gray')
    
    axes[k, 2].set_title("Predicted")
    axes[k, 2].imshow(im_result, cmap='gray')